# Advanced Querying Mongo

Importing libraries and setting up connection

In [4]:
import pandas as pd

In [5]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')

In [6]:
db= client.companies

In [7]:
col= db.coleccion

In [8]:
df= pd.DataFrame(list(col.find()))

In [9]:
df.head(2)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [10]:
# Your Code

query={'name': 'Babelgum'}

select={'_id': 0, 'name': 1}

list(col.find(query, select))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [11]:
# Your Code
query={'number_of_employees': {'$gt': 5000}}

In [12]:
pipeline = [{'$match': query},
            {'$project': {'_id': 0,
                        'Name': '$name',
                        'Employees': '$number_of_employees' }},
            {'$sort': {'employees': -1}}, {'$limit': 5}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Employees
0,Yahoo!,13600
1,Google,28000
2,eBay,15000
3,Facebook,5299
4,Cisco,63000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [13]:
# Your Code
query={'$and': [{'founded_year':  {'$gte': 2000}},
                {'founded_year':  {'$lte': 2005}}] }


pipeline = [{'$match': query},
            {'$project': {'_id': 0,
                        'Name': '$name',
                        'Founded year': '$founded_year' }}, {'$limit': 5}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Founded year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [14]:
# Your Code
query={ '$and': [{'ipo.valuation_amount':  {'$gt': 100000000}},
            {'founded_year':  {'$lte': 2010}}] }


pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'IPO': '$ipo' ,
                        'Valuation amount': '$valuation_amount',
                        'Valuation currency code': '$valuation_currency_code',
                        'Pub year': '$pub_year',
                        'Pub Month': '$pub_month',
                        'Stock Symbol': '$stock_symbol',
                        'Publication year': {'$concat': ['$pub_year', ' - ', '$pub_month']}}}, { '$limit': 3}]

#A que queda preciosa con el normalize? :D 
pd.json_normalize(list(col.aggregate(pipeline)))

,Name,Publication year,IPO.valuation_amount,IPO.valuation_currency_code,IPO.pub_year,IPO.pub_month,IPO.pub_day,IPO.stock_symbol
0,Facebook,None,1.040000e+11,USD,2012,5,18,NASDAQ:FB
1,Twitter,None,1.810000e+10,USD,2013,11,7,NYSE:TWTR
2,Yelp,None,1.300000e+09,USD,2012,3,2,NYSE:YELP


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [15]:
# Your Code
query={ '$and': [{'number_of_employees':  {'$lte': 1000}},
            {'founded_year':  {'$lte': 2005}}] }


pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Number of employees': '$number_of_employees' ,
                        'Founded Year': '$founded_year'}}, { '$limit': 10}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Number of employees,Founded Year
0,Wetpaint,47,2005
1,AdventNet,600,1996
2,Digg,60,2004
3,Fox Interactive Media,0,1979
4,Plaxo,50,2002
5,Technorati,35,2002
6,AddThis,120,2004
7,Jingle Networks,35,2005
8,Meetup,75,2002
9,LifeLock,644,2005


### 6. All the companies that don't include the `partners` field.

In [16]:
# Your Code
query={"partners":{"$exists":"false"}} 


pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name'}}, { '$limit': 5}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 7. All the companies that have a null type of value on the `category_code` field.

In [17]:
# Your Code
query={"category_code": { "$type": 10 }} 


pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Category Code':'$category_code'}}, { '$limit': 5}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Category Code
0,Collective,None
1,Snimmer,None
2,KoolIM,None
3,Level9 Media,None
4,VidKing,None


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [18]:
# Your Code
query={ '$and': [{'number_of_employees':  {'$lte': 1000}},
            {'number_of_employees':  {'$gt': 100}}] }


pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Number of employees': '$number_of_employees'}}, { '$limit': 5}, {'$sort': {'number_of_employees': -1}}] 

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Number of employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [19]:
# Your Code
pipeline = [{ '$project': {'_id': 0,
                        'Name': '$name'}}, { '$limit': 5}, {'$sort': {'IPO': -1}}] 

pd.DataFrame(list(col.aggregate(pipeline)))

,Name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [20]:
# Your Code
query={ 'number_of_employees':  {'$gte': 1}}
pipeline = [{ '$match': query},{ '$project': {'_id': 0,
                        'Name': '$name',
                        'Number of employees': '$number_of_employees'}}, { '$limit': 5}, {'$sort': {'Number of employees': -1}}] 

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Number of employees
0,Facebook,5299
1,Zoho,1600
2,AdventNet,600
3,Digg,60
4,Wetpaint,47


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [21]:
# Your Code
query={ '$and': [{'founded_month':  {'$gt': 6}},
            {'founded_month':  {'$lte': 12}}] }

pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Founded Month': '$founded_month' ,
                        'Founded Year': '$founded_year'}}, { '$limit': 10}, { '$sort': {'Founded Year': -1}}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Founded Month,Founded Year
0,Joost,10,2006
1,Powerset,10,2006
2,Kyte,12,2006
3,Thoof,12,2006
4,Wetpaint,10,2005
5,Zoho,9,2005
6,Omnidrive,11,2005
7,Digg,10,2004
8,Plaxo,11,2002
9,eBay,9,1995


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [43]:
# Your Code
query={ '$and': [{'founded_year':  {'$lte': 2000}},
            {'acquisition.price_amount':  {'$gte': 100000}}] }

pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Founded Year': '$founded_year',
                        'Acquisition Amount': '$acquisition.price_amount'}}, { '$limit': 10}, { '$sort': {'Acquisition Amount': 1}}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Founded Year,Acquisition Amount
0,Cyworld,1999,7.140000e+06
1,Recipezaar,1999,2.500000e+07
2,Neopets,1999,1.600000e+08
3,SideStep,1999,1.800000e+08
4,Snapfish,1999,3.000000e+08
5,Postini,1999,6.250000e+08
6,Alibaba,1999,1.000000e+09
7,Zappos,1999,1.200000e+09
8,PayPal,1998,1.500000e+09
9,Sun Microsystems,1982,7.400000e+09


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [44]:
# Your Code
query={ '$and': [{'acquisition.acquired_year':  {'$gte': 2010}},
            {'acquisition.price_amount':  {'$ne': None}}] }



pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Acquisition': '$acquisition'}
            }, { '$limit': 5}, { '$sort': {'Acquisition.price_amount': 1}}
           ]

pd.json_normalize(list(col.aggregate(pipeline)))

,Name,Acquisition.price_amount,Acquisition.price_currency_code,Acquisition.term_code,Acquisition.source_url,Acquisition.source_description,Acquisition.acquired_year,Acquisition.acquired_month,Acquisition.acquired_day,Acquisition.acquiring_company.name,Acquisition.acquiring_company.permalink
0,Digg,500000,USD,None,http://techcrunch.com/2012/07/12/betaworks-acq...,Betaworks Acquires Digg (TechCrunch),2012,7,12,betaworks,betaworks
1,Wetpaint,30000000,USD,cash_and_stock,http://allthingsd.com/20131216/viggle-tries-to...,Viggle Tries to Bulk Up Its Social TV Busines...,2013,12,16,Viggle,viggle
2,Revision3,30000000,USD,None,http://allthingsd.com/20120503/discovery-gets-...,"Discovery Gets a Web Video Arm, Courtesy of Re...",2012,5,3,Discovery Communications,discoverycommunications
3,Jingle Networks,62500000,USD,cash_and_stock,http://www.masshightech.com/stories/2011/04/11...,Jingle Networks bought for up to $62.5M,2011,4,11,Marchex,marchex
4,iContact,169000000,USD,cash_and_stock,http://techcrunch.com/2012/02/28/vocus-buys-em...,Vocus Buys Email Marketing Company iContact Fo...,2012,2,28,Vocus,vocus


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [45]:
# Your Code
query={'founded_year':  {'$gt': 0}}

pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Founded Year': '$founded_year'}}, { '$limit': 5}, { '$sort': {'Founded Year': 1}}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Founded Year
0,AdventNet,1996
1,Digg,2004
2,Facebook,2004
3,Wetpaint,2005
4,Zoho,2005


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [46]:
# Your Code
query={ '$and': [{'founded_day':  {'$lte': 7}},
                {'acquisition.price_amount':  {'$gt': 0}}]}



pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Acquisition Price': '$acquisition.price_amount'}
            }, { '$limit': 5}, { '$sort': {'Acquisition Price': -1}}
           ]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Acquisition Price
0,Postini,625000000
1,Jajah,207000000
2,Jingle Networks,62500000
3,Helio,39000000
4,StumbleUpon,29000000


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [47]:
# Your Code
query = {'$and': [{'category_code': 'web'},
                  {'number_of_employees': {'$gte': 4000}}]}

pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Number of employees': '$number_of_employees',
                        'Category': '$category_code' }
            }, { '$limit': 5}, { '$sort': {'Number of employees': 1}}
           ]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Number of employees,Category
0,AOL,8000,web
1,Webkinz,8657,web
2,Rakuten,10000,web
3,Yahoo!,13600,web
4,eBay,15000,web


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [48]:
# Your Code
query = {'$and': [{'acquisition.price_amount': {'$gte': 10000000}},
                  {'acquisition.price_currency_code': 'EUR'}]}

pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Acquisition Price Amount': '$acquisition.price_amount',
                        'Acquisition Price Currency Code': '$acquisition.price_currency_code' }
            }, { '$limit': 5}, { '$sort': {'Acquisition Price Amount': 1}}
           ]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Acquisition Price Amount,Acquisition Price Currency Code
0,Wayfinder,24000000,EUR
1,ZYB,31500000,EUR
2,Greenfield Online,40000000,EUR
3,Webedia,70000000,EUR
4,Apertio,140000000,EUR


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [49]:
# Your Code
query={'$and': [{'acquisition.price_amount': {'$gt': 0}},
                  {'acquisition.acquired_month':  {'$lte': 3}}]}




pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Acquisition': '$acquisition'}}, { '$limit': 5}, { '$sort': {'Acquisition.acquired_year': 1}}
           ]

pd.json_normalize(list(col.aggregate(pipeline)))

,Name,Acquisition.price_amount,Acquisition.price_currency_code,Acquisition.term_code,Acquisition.source_url,Acquisition.source_description,Acquisition.acquired_year,Acquisition.acquired_month,Acquisition.acquired_day,Acquisition.acquiring_company.name,Acquisition.acquiring_company.permalink
0,NetRatings,327000000,USD,cash,http://login.vnuemedia.com/hr/login/login_subs...,Nielsen buys rest of NetRatings,2007,2,NaN,Nielsen,nielsen
1,Revver,5000000,USD,None,http://newteevee.com/2008/02/14/liveuniverse-b...,LiveUniverse Buys Revver for More than a Song ...,2008,2,14.0,LiveUniverse,liveuniverse
2,Pluck,75000000,USD,cash,http://www.techcrunch.com/2008/03/04/demand-me...,Demand Media Buys Pluck for $75 million,2008,3,4.0,Demand Media,demandmedia
3,iContact,169000000,USD,cash_and_stock,http://techcrunch.com/2012/02/28/vocus-buys-em...,Vocus Buys Email Marketing Company iContact Fo...,2012,2,28.0,Vocus,vocus
4,Dailymotion,168000000,USD,None,http://techcrunch.com/2013/02/22/orange-acquir...,Orange Acquired Dailymotion For $168 Million B...,2013,2,22.0,Orange,orange


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [50]:
# Your Code
query={'$and': [{'founded_year': {'$gte': 2000}},
                {'founded_year': {'$lte': 2010}},
                {'acquisition.acquired_year':  {'$gte': 2011}}]}


pipeline = [{ '$match': query},
            { '$project': {'_id': 0,
                        'Name': '$name',
                        'Founded Year': '$founded_year'}}, { '$limit': 5}, { '$sort': {'Founded Year': 1}}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Founded Year
0,Digg,2004
1,Wetpaint,2005
2,Jingle Networks,2005
3,Geni,2006
4,Kyte,2006


### 20. All the companies that have been 'deadpooled' after the third year.

In [51]:
query = {
    '$and': [
        {'deadpooled_year': {'$gte': 1}},
        {'$expr': {'$gte': [{'$subtract': ['$deadpooled_year', '$founded_year']}, 3]}}
    ]
}

In [52]:
# Your Code

pipeline = [{ '$match': query},
            { '$project': 
             {'_id': 0,
                        'Name': '$name',
                        'Founded Year': '$founded_year'}}, { '$limit': 5}, { '$sort': {'Founded Year': 1}}]

pd.DataFrame(list(col.aggregate(pipeline)))

,Name,Founded Year
0,Omnidrive,2005
1,Wesabe,2005
2,Jangl SMS,2005
3,Thoof,2006
4,Babelgum,2007
